In [17]:
#https://github.com/datablogger-ml/Time-Series-Forecasting/blob/master/Time%20Series%20Forecasting.ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab 
from pylab import *
pylab.rcParams['figure.figsize'] = (16, 6) # size for all figures
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings

In [18]:
df = pd.read_csv(r'C:\Users\91783\Desktop\Results\YJ_Mean_Date_PM2.csv')
df.head()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

#y = pd.Series(data=df['PM2.5'].values, index=df['Date'])
#y.head()

,Date,PM2.5
0,2021-01-01,1.575013
1,2021-02-01,1.127264
2,2021-03-01,0.042478
3,2021-04-01,-0.261624
4,2021-05-01,0.116524


In [19]:
df = df.set_index('Date').asfreq('D')
df.columns = ['PM2.5']

In [20]:
print(f'The Daily data consists of values from {df.index.min()} to {df.index.max()}')

The Daily data consists of values from 2021-01-01 00:00:00 to 2021-12-31 00:00:00


In [21]:
#from pmdarima import auto_arima
filterwarnings('ignore')

In [22]:
n_test = 2
train = df.iloc[:len(df) - n_test]
test = df.iloc[len(df) - n_test:]
print(train.shape)
print(test.shape)


(363, 1)
(2, 1)


In [23]:
train_array = train["PM2.5"]
print(train_array.shape)

test_array = test["PM2.5"]
print(test_array.shape) 

(363,)
(2,)


In [24]:
from statsmodels.tsa.stattools import adfuller

print("p-value:", adfuller(train_array.dropna())[1])

p-value: 0.4158387613177612


In [25]:
diff_1 = train_array.diff().dropna()
print("p-value:", adfuller(diff_1.dropna())[1])

p-value: 5.0015359837271354e-23


In [26]:
import pmdarima as pmd

def arimamodel(timeseriesarray):
    autoarima_model = pmd.auto_arima(timeseriesarray, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              trace=True)
    return autoarima_model


In [27]:
arima_model = arimamodel(train_array)
arima_model.summary()

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=76.227, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=80.952, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=82.089, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=81.023, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=79.002, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=46.261, Time=0.18 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=52.951, Time=0.06 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=44.320, Time=0.25 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=53.603, Time=0.07 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=45.255, Time=0.39 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=45.893, Time=0.39 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=44.662, Time=0.23 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=53.861, Time=0.09 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=47.823, Time=0.56 sec
 ARIMA(3,1,1)(0,0,0)[0]             : AIC=42.336, Time=0.11 s

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  363
Model:               SARIMAX(3, 1, 1)   Log Likelihood                 -16.168
Date:                Thu, 16 Jun 2022   AIC                             42.336
Time:                        12:03:45   BIC                             61.794
Sample:                             0   HQIC                            50.071
                                - 363                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8585      0.085     10.066      0.000       0.691       1.026
ar.L2         -0.3349      0.061     -5.491      0.000      -0.454      -0.215
ar.L3          0.1351      0.083      1.630      0.103      -0.027       0.298
ma.L1         -0.8572      0.082    -10.508      0.000      -1.017      -0.697
sigma2         0.0639      0.003     18.522      0.000       0.057       0.071
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):               200.14
Prob(Q):                              0.88   Prob(JB):                         0.00
Heteroskedasticity (H):               1.66   Skew:                            -0.58
Prob(H) (two-sided):                  0.01   Kurtosis:                         6.46
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""